In [ ]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

In [1]:
# Soudamini Kidambi: 202003011
# Kanishk Menaria: 202003018
# Divyansh Jain: 202003047
import boto3 #python SDK for Amazon AWS
#Importing libraries
from braket.circuits import Circuit
from braket.aws import AwsDevice
from braket.devices import LocalSimulator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Initialize the simulator
device = LocalSimulator()

# Creating function for 3-qubit CNOT gate
def three_control_not(circuit):
    circuit.ccnot(0, 1, 3)
    circuit.ccnot(2, 3, 4)
    circuit.cnot(4, 5)
    circuit.ccnot(2, 3, 4)
    circuit.ccnot(0, 1, 3)
    return circuit

In [3]:
# Print circuit for 3-qubit CNOT gate to check
circuit = Circuit()
circuit = three_control_not(circuit)
print("Three-Control qubit NOT Gate:\n\n", circuit)
circuit.draw()

Three-Control qubit NOT Gate:

 T  : |0|1|2|3|4|
                
q0 : -C-------C-
      |       | 
q1 : -C-------C-
      |       | 
q2 : -|-C---C-|-
      | |   | | 
q3 : -X-C---C-X-
        |   |   
q4 : ---X-C-X---
          |     
q5 : -----X-----

T  : |0|1|2|3|4|


AttributeError: 'Circuit' object has no attribute 'draw'

In [4]:
# Increment circuit using 3 CNOT, 2 CNOT, and 1 CNOT gate
def increment_circuit(circuit):
    circuit = three_control_not(circuit)
    circuit.ccnot(0, 1, 2)
    circuit.cnot(0, 1)
    return circuit

In [5]:
# Decrement circuit using 3 CNOT, 2 CNOT, and 1 CNOT gate
def decrement_circuit(circuit):
    circuit.x([0, 1, 2])
    circuit = increment_circuit(circuit)
    circuit.x([0, 1, 2])
    return circuit

In [6]:
# Initialize combined circuit consisting of increment and decrement circuits
def init_circuit(init_state):
    circuit = Circuit()

    # Apply initial state
    if init_state[0] == '1':
        circuit.x(5)
    if init_state[1] == '1':
        circuit.x(2)
    if init_state[2] == '1':
        circuit.x(1)

    circuit.h(0)  # Hadamard gate at 0 qubit

    circuit = increment_circuit(circuit)
    circuit = decrement_circuit(circuit)

    return circuit

In [7]:
# Initial state = <q5 q2 q1>, <q3 q4> = extra qubits.
# <q0> = controller qubit for increment_circuit and decrement_circuit.
init_state = "011"
circuit = init_circuit(init_state)
print(circuit)

T  : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|
                                               
q0 : -H-C-------C-C-C-X-C----------C--C--C--X--
        |       | | |   |          |  |  |     
q1 : -X-C-------C-C-X-X-C----------C--C--X--X--
        |       | |     |          |  |        
q2 : -X-|-C---C-|-X-X---|-C-----C--|--X--X-----
        | |   | |       | |     |  |           
q3 : ---X-C---C-X-------X-C-----C--X-----------
          |   |           |     |              
q4 : -----X-C-X-----------X--C--X--------------
            |                |                 
q5 : -------X----------------X-----------------

T  : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|


In [8]:
# Initialize the initial state
next_state = init_state

# Create an empty list to store the path
path = []

# Repeat the circuit application and measurement process
for i in range(1):
    # Create an empty list to store the intermediate states
    intermediate_path = []

    # Apply the circuit and measure the output state 1000 times
    for j in range(1000):
        # Create a new quantum circuit for the current state
        circuit = init_circuit(next_state)

        # Run the circuit on the device and get the measurement results
        result = device.run(circuit, shots=1).result()
        measurement_counts = result.measurement_counts

        # Extract the next state from the measurement results
        for x in measurement_counts:
            next_state = x
        next_state = next_state[5] + next_state[2:0:-1]  # Measure the 5th, 2nd, and 1st qubits and store the result

        # Append the next state to the intermediate path
        intermediate_path.append(next_state)

    # Append the intermediate path to the main path
    path.append(intermediate_path)

In [9]:
# Import the CSV module
import csv

# Open the CSV file for writing
with open('path.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object with space as the delimiter
    my_writer = csv.writer(csvfile, delimiter=' ')
    
    # Write the data to the CSV file row by row
    for row in path:
        my_writer.writerow(row)

# Print the path to the console
print(path)


[['010', '001', '010', '011', '010', '011', '010', '011', '010', '001', '000', '001', '000', '111', '000', '001', '010', '011', '010', '001', '010', '011', '100', '011', '100', '101', '110', '101', '100', '101', '110', '101', '100', '101', '100', '011', '010', '001', '010', '001', '010', '011', '100', '101', '100', '101', '110', '111', '000', '111', '000', '001', '000', '111', '000', '001', '010', '011', '010', '011', '100', '011', '010', '011', '100', '101', '100', '101', '100', '101', '100', '101', '110', '111', '000', '111', '000', '001', '010', '001', '010', '001', '000', '111', '000', '111', '000', '001', '000', '111', '110', '111', '000', '111', '000', '111', '110', '101', '100', '101', '100', '011', '010', '011', '100', '101', '100', '011', '100', '101', '100', '101', '110', '111', '110', '101', '110', '101', '100', '011', '100', '101', '110', '111', '000', '001', '010', '011', '100', '011', '010', '011', '100', '101', '110', '111', '000', '001', '000', '001', '000', '001', '010

If you're like many people who learned quantum computing in the past several years, you might have learned how to program quantum circuits with [Qiskit](https://qiskit.org): the open-source quantum Software Development Kit (SDK) first released in 2017. With the [Qiskit-Braket provider](https://github.com/qiskit-community/qiskit-braket-provider/blob/main/docs/tutorials/0_tutorial_qiskit-braket-provider_overview.ipynb), you can run your Qiskit code across any of the gate-based devices on the [Amazon Braket](https://aws.amazon.com/braket/) quantum computing service.

**Note**: if you're running this in your local development environment (i.e. not from the Braket console), you'll need to make sure you've got your AWS account configured properly first to access Braket devices. Check out [this tutorial](https://aws.amazon.com/blogs/quantum-computing/setting-up-your-local-development-environment-in-amazon-braket/) for a walkthrough.

## Access Braket devices from Qiskit 

 There are quite a few different backend devices on Amazon Braket, so we'll walk through them one by one and give an example of recommended use cases for each.

### Quantum simulators
Let's start with the ***local simulator***. This is a quantum full state vector simulator which runs *locally* -- that means wherever you're running this Jupyter notebook (e.g. your local development environment or a notebook instance on the Braket console).

**Recommended use case:** Noiseless circuits up to ~12 qubits

In [ ]:
from qiskit_braket_provider import BraketLocalBackend

local_simulator = BraketLocalBackend()

Next, we have the ***local density matrix simulator***. This simulator also runs on your local machine, but allows you to simulate the effects of *noise* on your quantum circuit. Because density matrices are twice the size of state vectors, the number of qubits you can effectively simulate is half the size as the local state vector simulator.

**Recommended use case:** Noisy circuits up to ~6 qubits

In [ ]:
local_dm_simulator = BraketLocalBackend(name='braket_dm')

Now let's look at Braket's *on-demand* simulators: these run on AWS computing resources and have some expanded features in addition to those of the local simulator. We can list all the available Braket simulators with the following code:

In [ ]:
from qiskit_braket_provider import AWSBraketProvider

provider = AWSBraketProvider()

provider.backends(statuses=["ONLINE"], types=["SIMULATOR"])

First up for on-demand simulators is ***SV1***. This is a full state vector simulator which allows you to simulate larger circuits than the local simulator, along with the ability to [batch tasks](https://docs.aws.amazon.com/braket/latest/developerguide/braket-batching-tasks.html) and run them in parallel, as well as use advanced techniques like [adjoint gradient calculations](https://pennylane.ai/blog/2022/12/computing-adjoint-gradients-with-amazon-braket-sv1/) for variational quantum algorithms. 

**Recommended use case:** Noiseless variational algorithms on up to 34 qubits

In [ ]:
aws_sv1 = provider.get_backend("SV1")

The next on-demand simulator is ***DM1***. This is a density matrix simulator which, like SV1, allows you to simulate a larger number of qubits, as well as take advantage of batch execution. 

**Recommended use case:** Noisy variational algorithms on up to 17 qubits

In [ ]:
aws_dm1 = provider.get_backend("dm1")

Lastly for on-demand simulators, we have ***TN1***. This is a tensor-network simulator, which represents each gate in a circuit as a tensor. TN1 can simulate a larger number of qubits for circuits with local gates or other special structure, but will typically be slower than SV1 or DM1 for circuits with long-range or all-to-all gate structure.

**Recommended use case:** Noiseless quantum circuits with local connectivity and up to 50 qubits

**Note**: Each AWS resource, (like a file or CPU or QPU) lives in a specific region and may only be accessible from that region. For example, TN1 is only available in the `eu-west-2`, `us-east-1`, and `us-west-2` regions. 

To change your AWS region if you're running in a managed notebook, you'll need to use the GUI in the top right hand corner of the AWS console to select your new region, then relaunch or create your notebook from the Braket console.

To change your AWS region if you're running in your local development environment, you run the following code snippet:
```
import os
os.environ["AWS_REGION"] = "your-desired-region"
```

In [ ]:
# If you've switched to one of the regions where TN1 is accessible, feel free to uncomment the following code
# aws_tn1 = provider.get_backend("TN1")

### Quantum Processing Units (QPUs)

Amazon Braket also provides access to a number of third-party quantum hardware devices. The following code shows how to view the supported QPUs which are currently online:

In [ ]:
# provider.backends(statuses=["ONLINE"], types=["QPU"])

For a closer look at each quantum computer, you can peruse the [Providers Overview](https://aws.amazon.com/braket/quantum-computers/) on the Braket homepage, or the Devices tab on the left side of the Braket console. 
Currently only gate-based QPUs (IonQ, OQC, Rigetti) are supported with the Qiskit-Braket provider. 

## Running circuits on Braket devices



Now that we've walked through each of the quantum devices available through the Qiskit-Braket provider, let's take them for a spin! For this example, we'll create a 3-GHZ state on the Rigetti device, but feel free to choose a different QPU from the commented out devices below.

In [ ]:
qpu_backend = provider.get_backend("Aspen-M-3")
# qpu_backend = provider.get_backend("Lucy")
# qpu_backend = provider.get_backend("IonQ Device")

print(qpu_backend)

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(0, 2)
circuit.draw()

In [ ]:
# run circuit
qpu_task = qpu_backend.run(circuit, shots=10)

Each task you run is assigned a unique ARN (Amazon Resource Name), which you can save and use to retrieve the data for your task after its run, even if you close your notebook.

In [ ]:
task_id = qpu_task.job_id()
task_id

**Note**: Qiskit uses the name \"job\" to refer to what is called a \"task\" in Amazon Braket. This is why you access the ARN for the task by calling `.job_id()`.  

Now, Braket has a separate feature called \"Hybrid *Jobs*\", which is beyond the scope of this notebook, but which you can read about in the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html).

In [ ]:
# Retrieve task data
retrieved_task = qpu_backend.retrieve_job(job_id=task_id)

Then you can check the status of the task to see if it's finished:

In [ ]:
retrieved_task.status()

**Note:** different devices may have different availability windows, so while your task may not run right away, rest assured it will be added to the queue to be run when the device is back online.

When your task is finished, you can retrieve the data:

In [ ]:
data = retrieved_task.result()

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(data.get_counts())

## Running algorithms on Braket devices

You can also use the Qiskit-Braket provider to run built-in Qiskit algorithms on Braket backends. For example, we can run the VQE algorithm to find the ground state of hydrogen. We'll use the local simulator since the problem can be expressed in a basis that only requires a few qubits and will run quickly.

In [ ]:
from qiskit.opflow import (
    I,
    X,
    Z,
)

# Define the Hamiltonian operator for H2 in terms of Pauli spin operators
H2_op = (
    (-1.052373245772859 * I ^ I)
    + (0.39793742484318045 * I ^ Z)
    + (-0.39793742484318045 * Z ^ I)
    + (-0.01128010425623538 * Z ^ Z)
    + (0.18093119978423156 * X ^ X)
)

In [ ]:
# Import some utilities
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

# Define a `QuantumInstance` with a Braket backend
qi = QuantumInstance(local_simulator, seed_transpiler=42, seed_simulator=42)

# Specify VQE configuration
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi)

# Find the ground state
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

## What now?

The sky is the limit! Keep in mind, the Qiskit-Braket provider is still new and experimental, so if you run into a bug or want a new feature supported, consider [submitting a GitHub issue](https://github.com/qiskit-community/qiskit-braket-provider/issues) and opening a feature branch to join in on the development effort yourself!

In [ ]:
print("Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.2f} USD")